In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy
import pickle
from glob import glob
from typing import Any, Dict, List, Tuple, Union
import pandas as pd

"""Change to the data folder"""
new_path = "./new_train/new_train"
test_path = './new_val_in/new_val_in'
subset_test_path = './new_train/train_subset'
# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
test_dataset = ArgoverseDataset(data_path=test_path)

### Create a loader to enable batch processing

In [3]:
batch_sz = 5

def train_agents_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:],scene['v_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    out = numpy.concatenate([numpy.dstack([scene['p_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:],scene['v_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]

def train_all_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:],scene['v_in'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:]]) for scene in batch])     
    out = numpy.concatenate([numpy.dstack([scene['p_out'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:],scene['v_out'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]
    
def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:],scene['v_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    idx = [numpy.dstack([scene['scene_idx']]) for scene in batch]
    return inp, idx
    
train_agent_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_agents_collate, num_workers=0)

train_all_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_all_collate, num_workers=0)

test_loader = DataLoader(test_dataset,batch_size=batch_sz, shuffle = True, collate_fn=test_collate, num_workers=0)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class EncoderRNN(nn.Module):
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    def __init__(self,
                 input_size = 4,
                 embedding_size = 16,
                 hidden_size = 32):
        
        super(EncoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.linear = nn.Linear(input_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear(x))
        hidden = self.lstm(embedded, hidden)
        return hidden


class DecoderRNN(nn.Module):
    """Decoder Network."""
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    def __init__(self, embedding_size=16, hidden_size=32, output_size=4):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.linear1 = nn.Linear(output_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        x = self.linear1(x)
        embedded = F.relu(x)
        hidden = self.lstm(embedded, hidden)
        output = self.linear2(hidden[0])
        return output, hidden


In [5]:
from tqdm import tqdm_notebook as tqdm

def train(encoder, decoder, device, train_loader, encoder_optimizer, decoder_optimizer, epoch, log_interval=10000):    
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    iterator = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    criterion = nn.MSELoss()
    
    for i_batch, sample_batch in enumerate(iterator):
        
        inp, out = sample_batch
#         print(inp.shape)
        # preprocessing more ????
#         inp = inp[:,0,:,:]
#         out = out[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(inp.shape[0]):
            x_offset.append(inp[i][0][0].detach().clone())
            y_offset.append(inp[i][0][1].detach().clone())
    
        for j in range(inp.shape[0]):
            for i in range(inp.shape[1]):
                inp[j][i][0] = inp[j][i][0] - x_offset[j]
                inp[j][i][1] = inp[j][i][1] - y_offset[j]

        #output whatever
        for j in range(out.shape[0]):
            for i in range(out.shape[1]):
                out[j][i][0] = out[j][i][0] - x_offset[j]
                out[j][i][1] = out[j][i][1] - y_offset[j]
        
        _input, target = inp.to(device), out.to(device)
        
        encoder.train()
        decoder.train()
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        
        #encoder 
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        output_length = target.shape[1]
        feature_len = _input.shape[2]
        input_shape = _input.shape[2]
        
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
        
        loss = 0
        
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :]
        
        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden
#         print("DECODER INPUT", decoder_input.shape)
#         print("DECODER HIDDEN", decoder_hidden[0].shape)
        decoder_outputs = torch.zeros(target.shape).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Update loss
            loss += torch.sqrt(criterion(decoder_output[:, :4], target[:, di, :4]))

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        # Get average loss for pred_len
        loss = loss / 30

        # Backpropagate
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        
#         file1 = open("loss_steps.txt", "a")  # append mode
#         file1.write(str(loss.item()) + ",")
#         file1.close()
        
#       output = model(data)
#         loss = MSELoss(output, target)
        counter += 1
        if counter % (0.1*len(train_loader)) == 0:
            print("10% passed")
        iterator.set_postfix(loss=(loss.item()))
    

In [6]:
device = "cuda"
encoder = EncoderRNN(input_size=4)
decoder = DecoderRNN(output_size=4)

encoder = nn.DataParallel(encoder)
decoder = nn.DataParallel(decoder)

encoder.to(device)
decoder.to(device)

encoder_optimizer = torch.optim.Adam(encoder.parameters())
decoder_optimizer = torch.optim.Adam(decoder.parameters())

num_epoch = 1

for epoch in range(1, num_epoch + 1):
    train(encoder, decoder, device, train_all_loader, encoder_optimizer, decoder_optimizer, epoch)
#         train(encoder, decoder, device, train_agent_loader, encoder_optimizer, decoder_optimizer, epoch)
#         predict(model, device, test_loader)

In [7]:
def infer_absolute(
        test_loader: torch.utils.data.DataLoader,
        encoder: EncoderRNN,
        decoder: DecoderRNN,
#         start_idx: int,
#         forecasted_save_dir: str,
#         model_utils: ModelUtils,
):
    """Infer function for non-map LSTM baselines and save the forecasted trajectories.
    
    referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting
    
    Args:
        test_loader: DataLoader for the test set
        encoder: Encoder network instance
        decoder: Decoder network instance
        start_idx: start index for the current joblib batch
        forecasted_save_dir: Directory where forecasted trajectories are to be saved
        model_utils: ModelUtils instance

    """
    
    forecasted_trajectories = {}

    for i, (_input, idx) in enumerate(test_loader):
        
#         _input = _input[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(_input.shape[0]):
            x_offset.append(_input[i][0][0].detach().clone())
            y_offset.append(_input[i][0][1].detach().clone())
    
        for j in range(_input.shape[0]):
            for i in range(_input.shape[1]):
                _input[j][i][0] = _input[j][i][0] - x_offset[j]
                _input[j][i][1] = _input[j][i][1] - y_offset[j]

        _input = _input.to(device)

        # Set to eval mode
        encoder.eval()
        decoder.eval()

        # Encoder
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        input_shape = _input.shape[2]

        # Initialize encoder hidden state
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
       
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :4]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(
            (batch_size, 30, 4)).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        for i in range(30):
            for j in range(batch_size):
                decoder_outputs[j,i,0] = decoder_outputs[j,i,0] + x_offset[j]
                decoder_outputs[j,i,1] = decoder_outputs[j,i,1] + y_offset[j]
            
                if (idx[j][0][0][0] in forecasted_trajectories):
                    forecasted_trajectories[idx[j][0][0][0]].append(decoder_outputs[j,i,:].tolist())
                else:
                    forecasted_trajectories[idx[j][0][0][0]] = [decoder_outputs[j,i,:].tolist()]
                
    return(forecasted_trajectories)

In [8]:
output = infer_absolute(test_loader, encoder, decoder)

In [9]:
import pandas as pd
df = pd.DataFrame.from_dict(output, orient='index')
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
17586,"[246.46913146972656, 1300.831787109375, 1.1468...","[246.1080780029297, 1299.5404052734375, 1.0814...","[246.58253479003906, 1298.4456787109375, 1.049...","[246.69473266601562, 1297.183349609375, 0.9960...","[246.8292999267578, 1295.77392578125, 0.961817...","[246.93560791015625, 1294.382568359375, 0.9418...","[247.02960205078125, 1293.0531005859375, 0.928...","[247.16873168945312, 1291.8043212890625, 0.917...","[247.25657653808594, 1290.6162109375, 0.907122...","[247.33042907714844, 1289.4610595703125, 0.896...",...,"[248.7810516357422, 1275.10791015625, 1.010850...","[248.82012939453125, 1274.4482421875, 1.006687...","[248.51303100585938, 1273.1639404296875, 1.000...","[247.95614624023438, 1270.6002197265625, 1.014...","[248.6050262451172, 1268.677978515625, 1.08715...","[249.33810424804688, 1267.9365234375, 1.123092...","[249.75942993164062, 1267.4619140625, 1.108442...","[250.10581970214844, 1266.9737548828125, 1.100...","[250.23202514648438, 1266.2613525390625, 1.113...","[249.96585083007812, 1264.96142578125, 1.17100..."
7764,"[1871.2340087890625, 475.2296142578125, -10.16...","[1869.91845703125, 474.34173583984375, -10.445...","[1869.079345703125, 473.5448913574219, -10.498...","[1868.2989501953125, 472.78094482421875, -10.5...","[1867.4652099609375, 472.0090026855469, -10.50...","[1866.568603515625, 471.2337951660156, -10.498...","[1865.62353515625, 470.4592590332031, -10.4969...","[1864.6544189453125, 469.6867370605469, -10.49...","[1863.6353759765625, 468.9007568359375, -10.49...","[1862.6046142578125, 468.10260009765625, -10.4...",...,"[1853.093017578125, 459.0604248046875, -10.202...","[1852.5076904296875, 458.55438232421875, -10.1...","[1851.9072265625, 458.0321350097656, -10.15555...","[1851.08447265625, 457.262451171875, -10.13787...","[1849.6173095703125, 455.6693420410156, -10.12...","[1847.4183349609375, 453.253173828125, -10.106...","[1846.1744384765625, 452.2423095703125, -10.02...","[1845.5377197265625, 451.78448486328125, -9.99...","[1844.888427734375, 451.2497863769531, -10.013...","[1844.166748046875, 450.6512451171875, -10.052..."
11465,"[1830.128173828125, 442.6273498535156, -11.048...","[1829.21875, 442.01220703125, -11.304981231689...","[1828.3590087890625, 441.104248046875, -11.356...","[1827.4730224609375, 440.2732238769531, -11.38...","[1826.5357666015625, 439.45330810546875, -11.4...","[1825.543701171875, 438.63287353515625, -11.43...","[1824.5174560546875, 437.81707763671875, -11.4...","[1823.4869384765625, 437.0074768066406, -11.48...","[1822.4256591796875, 436.1876525878906, -11.49...","[1821.3729248046875, 435.3551330566406, -11.50...",...,"[1811.108642578125, 425.6591491699219, -11.213...","[1810.4759521484375, 425.1413879394531, -11.19...","[1809.552734375, 424.2812194824219, -11.183355...","[1807.6634521484375, 422.22906494140625, -11.1...","[1805.168212890625, 419.625732421875, -11.1671...","[1803.8104248046875, 418.59771728515625, -11.1...","[1802.91064453125, 417.9632873535156, -11.1180...","[1801.997802734375, 417.2880859375, -11.163330...","[1801.05126953125, 416.602783203125, -11.21698...","[1800.0858154296875, 415.9167175292969, -11.27..."
11305,"[421.1100769042969, 1148.70654296875, -2.72108...","[421.0726013183594, 1148.8526611328125, -2.667...","[420.4196472167969, 1149.2501220703125, -2.811...","[420.03802490234375, 1149.7069091796875, -2.91...","[419.7665710449219, 1150.1756591796875, -2.968...","[419.5169677734375, 1150.6068115234375, -3.002...","[419.2802734375, 1151.020751953125, -3.0250122...","[419.0312805175781, 1151.4544677734375, -3.042...","[418.78125, 1151.896240234375, -3.059004068374...","[418.5433044433594, 1152.3671875, -3.071511745...",...,"[415.8016052246094, 1157.308837890625, -3.1428...","[415.5545959472656, 1157.67333984375, -3.14832...","[415.3124694824219, 1158.027587890625, -3.1530...","[415.0293273925781, 1158.38720703125, -3.16316...","[414.7325134277344, 1158.7547607421875, -3.177...","[414.

In [10]:
for i in range(30):
    df[['v{}'.format((i*2)+1), 'v{}'.format((i*2)+2)]] = pd.DataFrame(df.get(i).tolist(), index=df.index)[[0, 1]]

In [11]:
import numpy as np
dropped_cols = list(np.arange(30))
df2 = df.drop(dropped_cols, axis=1)
df2.index.name = 'ID'
df2

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
ID,,,,,,,,,,,,,,,,,,,,,
17586,246.469131,1300.831787,246.108078,1299.540405,246.582535,1298.445679,246.694733,1297.183350,246.829300,1295.773926,...,249.338104,1267.936523,249.759430,1267.461914,250.105820,1266.973755,250.232025,1266.261353,249.965851,1264.961426
7764,1871.234009,475.229614,1869.918457,474.341736,1869.079346,473.544891,1868.298950,472.780945,1867.465210,472.009003,...,1847.418335,453.253174,1846.174438,452.242310,1845.537720,451.784485,1844.888428,451.249786,1844.166748,450.651245
11465,1830.128174,442.627350,1829.218750,442.012207,1828.359009,441.104248,1827.473022,440.273224,1826.535767,439.453308,...,1803.810425,418.597717,1802.910645,417.963287,1801.997803,417.288086,1801.051270,416.602783,1800.085815,415.916718
11305,421.110077,1148.706543,421.072601,1148.852661,420.419647,1149.250122,420.038025,1149.706909,419.766571,1150.175659,...,414.446167,1159.117676,414.173157,1159.470947,413.913818,1159.814209,413.667877,1160.147705,413.434875,1160.471680
38652,257.753235,1030.345215,257.437653,1029.191406,257.767792,1028.597656,257.912201,1028.035400,258.030365,1027.321045,...,259.385101,1006.718018,259.520782,1005.805298,259.654694,1005.045959,259.782227,1004.422180,259.901886,1003.910339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38919,198.537720,1522.380249,199.513824,1521.870605,200.488541,1522.253296,201.722107,1522.399658,203.032654,1522.622803,...,226.565048,1524.469482,227.374939,1524.290894,228.128433,1524.231323,228.825089,1524.265381,229.463074,1524.374512
36941,219.188934,1525.488892,218.803940,1525.166260,219.331772,1524.760498,219.891251,1524.476074,220.473785,1524.173584,...,232.959305,1520.551880,233.524475,1520.504639,234.064880,1520.454712,234.572388,1520.390381,235.043396,1520.303101
31538,739.843079,1174.994385,740.414307,1175.723267,740.412048,1176.239380,740.523438,1176.972412,740.609802,1177.700806,...,742.350647,1191.507568,742.413818,1192.087280,742.478516,1192.651855,742.545410,1193.201904,742.615112,1193.738159


In [12]:
df2.to_csv("outputs1epb5_v_all.csv", index=True)